# Market-implied stock price PDFs

Option prices embed information about the markets expectation of future performance of the underlying asset.
The set of European option prices across strikes for a given maturity $T$ implies a risk-neutral probability density function (details below) of the price $S_T$ of the underlying asset at the maturity.

This density reflects the distribution consistent with current option prices under the pricing measure that discounts payoffs at the risk-free rate.
It also reflects both how the market expects the underlying to move (although it is important to note that the pdf we obtain is NOT simply the market's prediction of where the stock is likely to move), as well as how the market prices risk across the various outcomes.

This project uses several years of daily SPX option chain data to extract the market-implied risk-neutral pdfs for options with 1 day maturities, 7 day maturities, and 28 day maturities.
Using the Breeden-Litzenberger relation, which says that the implied pdf is given by the second partial derivative of price at maturity with respect to strike:
$$ f_Q(K) = e^{rT} \frac{\partial^2 C(K, T)}{\partial K^2},$$
we numerically recover the pdf.
Since strikes come in discrete increments, techniques to interpolate in a suitable way are discussed.

We then discuss and compare ways of recovering the real-world probability distribution from the risk-neutral one.

There are two main goals of this project, and two secondary goals.
The main goals are
1. Compare the risk-neutral pdf to the realized price distribution.
2. Explore ways of estimating the real-world density function from the risk-neutral density.

The secondary goals are
* When it seems fruitful and interesting, I try to compare different numerical methods that arise during the process of accomplishing these goals.
* Give exposition of the theory and background needed to understand this topic so that a wider audience may find it useful and engaging.


## The data

The data I use was obtained from OptionsDX.
I have EOD options chain data for SPX for years 2020 through 2023.
I filtered for options with 1 day maturities, 7 day maturities, and 28 day maturities, so that I can compare all the methods across a range of expiration lengths.

To illustrate the idea, we'll just look at a single implied pdf from one collection of quotes.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from black_scholes import *

from scipy.interpolate import UnivariateSpline # for cubic interpolation with smoothing

In [2]:
spx_2020 = pd.read_csv("data/spx_options_2020.csv")

display(spx_2020)

,underlying_last,expire_unix,quote_unix,quote_time,call_iv,strike,call_bid,call_ask,put_bid,put_ask,dte,tte_unix
0,3002.74,1583355600,1583269200,2020-03-03 16:00,4.697070,1800.0,1200.81,1210.10,0.00,0.05,1.0,86400
1,3002.74,1583355600,1583269200,2020-03-03 16:00,,1900.0,1097.10,1107.30,0.00,0.05,1.0,86400
2,3002.74,1583355600,1583269200,2020-03-03 16:00,3.831740,1950.0,1050.80,1060.11,0.00,0.05,1.0,86400
3,3002.74,1583355600,1583269200,2020-03-03 16:00,,2000.0,995.10,1005.40,0.00,0.05,1.0,86400
4,3002.74,1583355600,1583269200,2020-03-03 16:00,,2050.0,945.60,955.79,0.00,0.05,1.0,86400
...,...,...,...,...,...,...,...,...,...,...,...,...
82083,3624.23,1609189200,1606770000,2020-11-30 16:00,0.344570,5000.0,0.00,0.10,1379.59,1389.40,28.0,2419200
82084,3624.23,1609189200,1606770000,2020-11-30 16:00,0.375080,5100.0,0.00,0.15,1465.20,1490.51,28.0,2419200
82085,3624.23,1609189200,1606770000,2020-11-30 16:00,0.365110,5200.0,0.00,0.04,1579.50,1589.30,28.0,2419200
82086,3624.23,1609189200,1606770000,2020-11-30 16:00,0.383380,5300.0,0.00,0.05,1679.51,1689.30,28.0,2419200


In [3]:

# one_m_calls = options_data[(options_data['expiration'] == '2025-11-28') & (options_data['type'] == 'call')].copy()
# two_m_calls = options_data[(options_data['expiration'] == '2025-12-19') & (options_data['type'] == 'call')].copy()
# one_m_puts = options_data[(options_data['expiration'] == '2025-11-28') & (options_data['type'] == 'put')].copy()
# two_m_puts = options_data[(options_data['expiration'] == '2025-12-19') & (options_data['type'] == 'put')].copy()

# display(one_m_calls)


In [4]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import datetime
# from black_scholes import *

# # load data and compute mid prices
# one_m_calls = pd.read_csv('data/spy_1m_calls.csv')
# one_m_calls['midPrice'] = (one_m_calls['bid'] + one_m_calls['ask']) / 2

# two_m_calls = pd.read_csv('data/spy_2m_calls.csv')

# # Read the trading price and date from the file
# with open('data/spy_trading_price.txt', 'r') as file:
#     trading_data = file.readlines()

# # Extract the trading price and date
# spy_price = float(trading_data[0].strip())
# trading_date = pd.to_datetime(trading_data[1].strip())

# expiry_date_1 = datetime.date(2025, 11, 28)
# print(expiry_date_1)
# expiry_date_2 = datetime.date(2025, 12, 19)

# # add a column for the time to expiry in years
# one_m_calls['timeToExpiry'] = one_m_calls['lastTradeDate'].apply(
#     lambda date_str: find_tte_yf_options('2025-11-28', pd.to_datetime(date_str))
# )

# print(f"SPY Trading Price: {spy_price}")
# print(f"Trading Date: {trading_date}")


# plt.figure(figsize=(10,6))
# # plt.plot(one_m_calls['strike'], one_m_calls['impliedVolatility'])
# plt.plot(one_m_calls['strike'], one_m_calls['lastPrice'])

In order to apply the formula that we want, we need to interpolate this to a smooth function.
Interpolating on prices directly is dangerous as it can lead to spurious arbitrage opportunities.
It makes more sense to convert these prices to IV using Black-Scholes, interpolate on IV, and then convert back.
(Note that this data does come with an implied volatility included, but I'll calculate my own so that I know exactly where it comes from.)


(TODO: I still don't quite understand why this is better.)

In [5]:
# from black_scholes import *

# one_m_calls['implied_volatility'] = one_m_calls.apply(
#     lambda row: implied_volatility_call(
#         market_price=row['lastPrice'],
#         S0 = row['spot_price'],
#         K=row['strike'],
#         t=row['time_to_expiration'],
#         r=0.0
#     ),
#     axis=1
# )

# # plt.plot(one_m_calls['strike'], one_m_calls['implied_volatility'], label='Implied Volatility (custom)')


# x = one_m_calls['strike']
# y = one_m_calls['implied_volatility']

# # Fit smoothing spline
# # s is the smoothing factor
# one_m_calls_iv_smoothed = UnivariateSpline(x, y, s=0.005)




# strikes_fine = np.linspace(x.min(), x.max(), 1000)
# iv_fine = one_m_calls_iv_smoothed(strikes_fine)

# plt.scatter(x, y, s=10, label='IVs')
# plt.plot(strikes_fine, iv_fine, 'r', label="Cubic fit", lw=2, alpha=0.5)
# plt.legend()
# plt.show()




# mean_strike = one_m_calls['spot_price'].mean()
# mean_tte = one_m_calls['time_to_expiration'].mean()

# bs_fine = np.zeros(1000)

# for i in range(1000):
#     bs_fine[i] = bs_call(S0 = mean_strike,
#                          K=strikes_fine[i],
#                         sigma=iv_fine[i],
#                         t=mean_tte,
#                         r=0.0
#     )


# # now convert back to prices using Black-Scholes
# # to keep it smooth, we need to use a single value for the spot price
# # I hope that isn't too weird
# # I'll use the average
# # visually they look basically the same if I use a different price nearby
# # so I hope it's okay
# # TODO: think about this
# one_m_calls['smoothed_price'] = one_m_calls.apply(
#     lambda row: bs_call(
#         # S0 = row['spot_price'],
#         S0 = one_m_calls['spot_price'].mean(),
#         K=row['strike'],
#         sigma=one_m_calls_iv_smoothed(row['strike']),
#         t=row['time_to_expiration'],
#         r=0.0
#     ),
#     axis=1
# )


# plt.plot(one_m_calls['strike'], one_m_calls['smoothed_price'], alpha=.5)
# plt.plot(x_fine, bs_fine)
# # plt.plot(one_m_calls['strike'], one_m_calls['lastPrice'], alpha=.5)
# plt.show()


Now we need to do a little numeric differentiation to apply the Breeden-Litzenberger formula, which says that
$$f(K) = e^{rt} \frac{\partial^2 C}{\partial K^2}$$


### The Breeden-Litzenberger Formula
Recall that the fair value of a call option with time $\tau$ until expiry is the present value of the expected payoff.
If $f$ denotes the PDF of the stock prices $S_T$ of some stock $S$ at time $T$, this is
$$
\begin{align*}
C(K, \tau) &= e^{-rt} \mathbb E(\max\{S_T - K, 0\}) \\
&= e^{-rt} \int_0^\infty \max\{S_T - K, 0\} f(S_T) \, d S_T \\
&= e^{-rt} \int_K^\infty (x - K) f(x) \, dx.
\end{align*}
$$
To derive the Breeden-Litzenberger formula, we just need to invert this to find $f$ as a function of $C$.
Taking the partial derivative with respect to $K$ gives
$$
\begin{align*}
\frac{\partial C}{\partial K} &= - e^{-rt} \int_{K}^\infty f(x) \, dx \\
&= e^{-rt} \left( \int_{-\infty}^K f(x) \, dx - 1 \right),
\end{align*}
$$
where in the second step we use the fact that $\int_{\R} f(x) \, dx = 1$.
But now we can find $f$ by taking one more derivative.
Doing this and rearranging, we see that
$$f(K) = e^{rt} \frac{\partial^2 C}{\partial K^2}.$$

In [6]:

# f = np.gradient(bs_fine, strikes_fine)
# f_2 = np.gradient(f, strikes_fine)

# fig, ax1 = plt.subplots()

# ax1.plot(x_fine, bs_fine)
# ax1.set_xlabel('Strike')
# ax1.set_ylabel('Price PDF (smoothed)', color='tab:blue')
# ax1.tick_params(axis='y', labelcolor='tab:blue')

# ax2 = ax1.twinx()
# ax2.plot(strikes_fine, f_2, color='tab:red', label='pdf')
# ax2.set_ylabel('f(K)', color='tab:red')
# ax2.tick_params(axis='y', labelcolor='tab:red')


So we see what the market thinks the price of SPY is likely to be in one month.
We also need to do this analysis for the puts, and for the calls and puts with 2-month expiries.

# References and further reading

[Estimating the real-world density from the option-implied risk-neutral density, a paper](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2093397)

[A blog post about market-implied probabilities](https://blog.thinknewfound.com/2017/11/market-implied-probabilities-useful/)